In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
from tqdm import tqdm

import itertools
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [2]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

# 파생변수 생성 및 인코딩

In [3]:
## 나이 양수로 변환
train['birth_plus'] = train.DAYS_BIRTH*(-1)/365
test['birth_plus'] = test.DAYS_BIRTH*(-1)/365

train['gender']=LabelEncoder().fit_transform(train.gender)
train['car']=LabelEncoder().fit_transform(train.car)
train['reality']=LabelEncoder().fit_transform(train.reality)
train['income_type']=LabelEncoder().fit_transform(train.income_type)
train['edu_type']=LabelEncoder().fit_transform(train.edu_type)
train['family_type']=LabelEncoder().fit_transform(train.family_type)
train['house_type']=LabelEncoder().fit_transform(train.house_type)
train.fillna('NAN',inplace=True)
train['occyp_type']=LabelEncoder().fit_transform(train.occyp_type)
train['income_weight']=train.income_total**2
train['income_age']=train.income_total*train.DAYS_BIRTH
train['income_emp']=train.income_total*train.DAYS_EMPLOYED

test['gender']=LabelEncoder().fit_transform(test.gender)
test['car']=LabelEncoder().fit_transform(test.car)
test['reality']=LabelEncoder().fit_transform(test.reality)
test['income_type']=LabelEncoder().fit_transform(test.income_type)
test['edu_type']=LabelEncoder().fit_transform(test.edu_type)
test['family_type']=LabelEncoder().fit_transform(test.family_type)
test['house_type']=LabelEncoder().fit_transform(test.house_type)
test.fillna('NAN',inplace=True)
test['occyp_type']=LabelEncoder().fit_transform(test.occyp_type)
test['income_weight']=test.income_total**2
test['income_age']=test.income_total*test.DAYS_BIRTH
test['income_emp']=test.income_total*test.DAYS_EMPLOYED

train.income_total=train.income_total.astype(int)
train.family_size=train.family_size.astype(int)
train.begin_month=train.begin_month.astype(int)
train.birth_plus=train.birth_plus.astype(int)

test.income_total=test.income_total.astype(int)
test.family_size=test.family_size.astype(int)
test.begin_month=test.begin_month.astype(int)
test.birth_plus=test.birth_plus.astype(int)

train.credit=train.credit.astype('category')
train.drop(['index','family_size','work_phone'],axis=1,inplace=True)
test.drop(['index','family_size','work_phone'],axis=1,inplace=True)
train.DAYS_BIRTH=train.DAYS_BIRTH**6
test.DAYS_BIRTH=test.DAYS_BIRTH**6
train.income_type=1/(train.income_type)
test.income_type=1/(test.income_type)
train.income_weight=1/(train.income_weight)**3
test.income_weight=1/(test.income_weight)**3
train['DAYS_BIRTH_weight']=1/(train.DAYS_BIRTH)**2
test['DAYS_BIRTH_weight']=1/(test.DAYS_BIRTH)**2

# Grid

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [5]:
# 그리드
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='accuracy',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=8, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [48]:
ftr=train.drop("credit",axis=1)
target=train['credit']


# 데이터 분할
X_train,X_val,y_train,y_val=train_test_split(ftr,target,test_size=0.25,random_state=40)


# 모델 객체 생성
model = HistGradientBoostingClassifier()


paramgrid = {'max_iter' : [1000],
            'learning_rate' : [0.08, 0.09, 0.1, 0.11],
             'max_depth' : [15,17,18,20,22,24,26, 28, 30],
             'max_leaf_nodes' : [26,28,30, 31, 32, 33, 34],
             'random_state' : [2021]
            }


model, pred = algorithm_pipeline(X_train, X_val, y_train, y_val, model, 
                                 paramgrid, cv=10, scoring_fit='accuracy')


print(model.best_score_)
print(model.best_params_)

Fitting 10 folds for each of 252 candidates, totalling 2520 fits
0.712277413260746
{'learning_rate': 0.08, 'max_depth': 18, 'max_iter': 1000, 'max_leaf_nodes': 33, 'random_state': 2021}


In [6]:
## predict_rf
ftr = train.drop('credit', axis=1).values
target = train['credit'].values
tst_ar = test.values
n_class = 3
n_fold = 10

In [7]:

print(ftr.shape)
print(tst_ar.shape)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=40)
 
hgb_p_val = np.zeros((ftr.shape[0], n_class))
hgb_p_tst = np.zeros((tst_ar.shape[0], n_class))
 
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    hgb_clf = HistGradientBoostingClassifier(max_iter=1000,
                                             learning_rate = 0.08,
                                                verbose=True,
                                             max_leaf_nodes = 33,
                                             max_depth = 18,
                                                random_state=2021
                                                )
    hgb_clf.fit(ftr[i_trn], target[i_trn])
    hgb_p_val[i_val, :] = hgb_clf.predict_proba(ftr[i_val])
    hgb_p_tst += hgb_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, hgb_p_val)}')
print(confusion_matrix(target, np.argmax(hgb_p_val, axis=1)))

(26457, 21)
(10000, 21)
training model for CV #1
Binning 0.004 GB of training data: 0.023 s
Binning 0.000 GB of validation data: 0.001 s
Fitting gradient boosted rounds:
[1/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.85811, val loss: 0.85893, in 0.027s
[2/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.84105, val loss: 0.84243, in 0.029s
[3/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.82802, val loss: 0.83105, in 0.026s
[4/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.81728, val loss: 0.82308, in 0.027s
[5/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.80813, val loss: 0.81526, in 0.028s
[6/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.80043, val loss: 0.80971, in 0.027s
[7/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.79320, val loss: 0.80414, in 0.027s
[8/1000] 3 trees, 99 leaves (33 on avg), max depth = 10, train loss: 0.78651, val

[76/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.62432, val loss: 0.73233, in 0.026s
[77/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.62252, val loss: 0.73198, in 0.027s
[78/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.62069, val loss: 0.73148, in 0.028s
[79/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.61908, val loss: 0.73125, in 0.027s
[80/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.61776, val loss: 0.73067, in 0.027s
[81/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.61622, val loss: 0.73020, in 0.028s
[82/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.61490, val loss: 0.72988, in 0.024s
[83/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.61310, val loss: 0.72927, in 0.027s
[84/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.61184, val loss: 0.72941, in 0.027s
[85/1000] 3 trees, 99 leaves

[152/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.52599, val loss: 0.71603, in 0.026s
[153/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.52506, val loss: 0.71587, in 0.025s
[154/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.52392, val loss: 0.71545, in 0.025s
[155/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.52281, val loss: 0.71536, in 0.025s
[156/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.52156, val loss: 0.71510, in 0.024s
[157/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.52067, val loss: 0.71476, in 0.025s
[158/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.51965, val loss: 0.71454, in 0.026s
[159/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.51841, val loss: 0.71436, in 0.026s
[160/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.51706, val loss: 0.71396, in 0.025s
[161/1000] 3 trees,

[39/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.68707, val loss: 0.75469, in 0.025s
[40/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.68496, val loss: 0.75405, in 0.024s
[41/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.68320, val loss: 0.75305, in 0.026s
[42/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.68094, val loss: 0.75238, in 0.025s
[43/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.67848, val loss: 0.75212, in 0.026s
[44/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.67643, val loss: 0.75166, in 0.025s
[45/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.67451, val loss: 0.75077, in 0.025s
[46/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.67257, val loss: 0.75040, in 0.025s
[47/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.67024, val loss: 0.74995, in 0.025s
[48/1000] 3 trees, 99 leaves

[115/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.56629, val loss: 0.72631, in 0.024s
[116/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.56494, val loss: 0.72588, in 0.024s
[117/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.56384, val loss: 0.72595, in 0.025s
[118/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.56293, val loss: 0.72612, in 0.023s
[119/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.56156, val loss: 0.72595, in 0.026s
[120/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.56044, val loss: 0.72565, in 0.025s
[121/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.55912, val loss: 0.72515, in 0.025s
[122/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.55796, val loss: 0.72526, in 0.025s
[123/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.55666, val loss: 0.72480, in 0.025s
[124/1000] 3 trees,

...

[159/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.51488, val loss: 0.72837, in 0.029s
[160/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.51378, val loss: 0.72852, in 0.027s
[161/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.51271, val loss: 0.72857, in 0.027s
[162/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.51150, val loss: 0.72836, in 0.027s
[163/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.51073, val loss: 0.72824, in 0.027s
[164/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.50977, val loss: 0.72809, in 0.028s
[165/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.50847, val loss: 0.72828, in 0.026s
[166/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.50744, val loss: 0.72811, in 0.023s
[167/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.50630, val loss: 0.72780, in 0.026s
[168/1000] 3 trees,

[38/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.69078, val loss: 0.75585, in 0.034s
[39/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.68805, val loss: 0.75462, in 0.032s
[40/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.68612, val loss: 0.75458, in 0.030s
[41/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.68395, val loss: 0.75409, in 0.029s
[42/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.68227, val loss: 0.75343, in 0.032s
[43/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.67985, val loss: 0.75271, in 0.030s
[44/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.67759, val loss: 0.75172, in 0.029s
[45/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.67601, val loss: 0.75161, in 0.027s
[46/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.67409, val loss: 0.75134, in 0.028s
[47/1000] 3 trees, 99 leaves

[36/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.69450, val loss: 0.76149, in 0.025s
[37/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.69268, val loss: 0.76058, in 0.024s
[38/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.69023, val loss: 0.76019, in 0.026s
[39/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.68785, val loss: 0.75892, in 0.027s
[40/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.68598, val loss: 0.75813, in 0.025s
[41/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.68360, val loss: 0.75721, in 0.026s
[42/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.68149, val loss: 0.75635, in 0.024s
[43/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.67961, val loss: 0.75546, in 0.027s
[44/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.67739, val loss: 0.75461, in 0.025s
[45/1000] 3 trees, 99 leaves

[112/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.56966, val loss: 0.71971, in 0.026s
[113/1000] 3 trees, 99 leaves (33 on avg), max depth = 16, train loss: 0.56862, val loss: 0.71959, in 0.025s
[114/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.56710, val loss: 0.71903, in 0.026s
[115/1000] 3 trees, 99 leaves (33 on avg), max depth = 15, train loss: 0.56597, val loss: 0.71896, in 0.024s
[116/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.56478, val loss: 0.71882, in 0.024s
[117/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.56351, val loss: 0.71822, in 0.025s
[118/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.56210, val loss: 0.71718, in 0.025s
[119/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.56086, val loss: 0.71664, in 0.024s
[120/1000] 3 trees, 99 leaves (33 on avg), max depth = 14, train loss: 0.55984, val loss: 0.71639, in 0.024s
[121/1000] 3 trees,

[188/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48701, val loss: 0.70543, in 0.025s
[189/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48587, val loss: 0.70465, in 0.029s
[190/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.48502, val loss: 0.70442, in 0.027s
[191/1000] 3 trees, 99 leaves (33 on avg), max depth = 17, train loss: 0.48414, val loss: 0.70454, in 0.025s
[192/1000] 3 trees, 99 leaves (33 on avg), max depth = 13, train loss: 0.48314, val loss: 0.70444, in 0.028s
[193/1000] 3 trees, 99 leaves (33 on avg), max depth = 18, train loss: 0.48205, val loss: 0.70420, in 0.027s
[194/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48124, val loss: 0.70429, in 0.026s
[195/1000] 3 trees, 99 leaves (33 on avg), max depth = 12, train loss: 0.48030, val loss: 0.70383, in 0.028s
[196/1000] 3 trees, 99 leaves (33 on avg), max depth = 11, train loss: 0.47958, val loss: 0.70371, in 0.027s
[197/1000] 3 trees,

In [56]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'hgbm'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, hgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, hgb_p_tst, fmt='%.6f', delimiter=',')

In [62]:
print(f'{log_loss(target, hgb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(hgb_p_val, axis=1))}%')

0.6811907370591721
[[  897   435  1890]
 [  221  2867  3179]
 [  414   782 15772]]%


In [ ]:
0.7193942496401385
5 fold 기준
    hgb_clf = HistGradientBoostingClassifier(max_iter=1000,
                                             learning_rate = 0.08,
                                                verbose=True,
                                             max_leaf_nodes = 33,
                                             max_depth = 18,
                                                random_state=2021
                                                )

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=40)
 
hgb_p_val = np.zeros((ftr.shape[0], n_class))
hgb_p_tst = np.zeros((tst_ar.shape[0], n_class))
 
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    hgb_clf = HistGradientBoostingClassifier(max_iter=1000,
                                             learning_rate = 0.08,
                                                verbose=True,
                                             max_leaf_nodes = 33,
                                             max_depth = 18,
                                                random_state=2021
                                                )

----

In [ ]:
sample_submission=pd.read_csv('data/sample_submission.csv')
for i in range(17):
    sample_submission.iloc[:,1:]+=models_lgbm[i].predict_proba(test)/17
    
sample_submission.to_csv('state2021.csv',index=False)

In [29]:
feature_lgbm=feature_lgbm.sort_index()
feature_lgbm

print(np.mean(np.array(logloss_avg)))

0.6880214218100895


In [30]:
print(np.mean(np.array(logloss_avg)))

0.6880214218100895


- 90 : 0.6890796591713713

----

In [28]:
############################################################################
label=train.credit
train0=train.drop('credit',axis=1)
skf=StratifiedKFold(n_splits=17,random_state=40,shuffle=True)
folds=[]
for train_idx,val_idx in skf.split(train0,label):
    folds.append([train_idx,val_idx])
models_lgbm={}
logloss_avg=[]
feature_lgbm=pd.DataFrame()
for i in range(17):
    train_x,val_x,train_y,val_y=train0.iloc[folds[i][0],:],train0.iloc[folds[i][1],:],label[folds[i][0]],label[folds[i][1]]
    lgb=LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=230, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=99,
                       verbose = 50
                       # reg_alpha=0.1
                    )
    lgb.fit(train_x,train_y)
    logloss_avg.append(log_loss(val_y,lgb.predict_proba(val_x)))
    models_lgbm[i]=lgb
    feature_lgbm=pd.concat([feature_lgbm,pd.DataFrame(lgb.predict_proba(val_x),index=folds[i][1])],axis=0)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.748574
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.247245
[LightGBM] [Debug] init for col-wise cost 0.001282 seconds, init for row-wise cost 0.001238 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [De

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Re-bagging, using 22402 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22422 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Deb

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22462 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22376 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22430 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22393 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22406 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Re-bagging, using 22414 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22472 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Traine

...

[LightGBM] [Debug] Re-bagging, using 22451 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22408 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22459 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22385 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22443 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debu

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22428 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Deb